# Principal Component Analysis

In [ ]:
import consts
import helper
import importlib
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import sklearn, scipy, requests
import random

In [ ]:
FILE_PATH = consts.RAW_DATA_PATH_RYAN

In [ ]:
importlib.reload(helper)
df = pd.read_csv(FILE_PATH + 'data.20211223_1200')
df = df[df.columns[8:]] # cut out the test data
helper.pca_plot(df, num_components=244)

In [ ]:
df = pd.read_csv(FILE_PATH + 'data.20200316_1200')
df.columns
df = df[df.columns[7:]] # cut out testing columns
helper.pca_plot(df, num_components=50)

## Regressions with PCA

In [ ]:
USER = "RYAN"
RESPONSE_NAME = consts.RESPONSE_NAME

LASSO = "LASSO"
XGBOOST = "XGBOOST"

ALPHA = 0.05 # Set significance level

TEST_START  = "20210202"
TRAIN_TEST_GAP = 31 # days

In [ ]:
importlib.reload(helper)
data = helper.Data(train_data_path=FILE_PATH)
# 6 months
train_df = data.update_and_get_train_df(TEST_START, backward_dayCount= 31, years_count = 0, train_data_count=182)
# Really the test_dfs are VALIDATION data
test_dfs = data.update_and_get_test_df(data_path = FILE_PATH, start_date=TEST_START, end_date="20210301")

transforming_cols = train_df.columns[7:]
y_test = train_df[RESPONSE_NAME]
train_df = train_df[transforming_cols]

In [ ]:
# Fit PCA
pca = sklearn.decomposition.PCA(n_components=15)
pca.fit(train_df)

In [ ]:
# Fit Transform X training data
train_df_pca = pd.DataFrame(pca.transform(train_df))
train_df_pca[RESPONSE_NAME] = y_test.values # leave untransformed

transformed_test = [pd.DataFrame(pca.transform(df[transforming_cols])) for df in test_dfs]
test_dfs_pca = []
for pca_df, df in zip(transformed_test, test_dfs):
    pca_df[RESPONSE_NAME] = df[RESPONSE_NAME]
    test_dfs_pca.append(pca_df)


In [ ]:
importlib.reload(helper)
full_ols_model = helper.Model('OLS')
full_ols_model.train(pd.DataFrame(train_df_pca))
predictions = full_ols_model.test(test_dfs_pca)

In [ ]:
importlib.reload(helper)
full_ols_model = helper.Model(LASSO)
full_ols_model.train(pd.DataFrame(train_df_pca))
predictions = full_ols_model.test(test_dfs_pca)

In [ ]:
importlib.reload(helper)
full_ols_model = helper.Model(XGBOOST)
full_ols_model.train(pd.DataFrame(train_df_pca))
predictions = full_ols_model.test(test_dfs_pca)